## Analizador Lexico
Un analizador lexico transforma una entrada en tokens

In [ ]:
entrada = "Es de dia entonces hace sol"

In [ ]:
tokens = entrada.lower().split()
tokens

['si',
 'si',
 'es',
 'de',
 'dia',
 'entonces',
 'mañana',
 'no',
 'es',
 'de',
 'noche']

In [ ]:
from enum import Enum

class TipoToken(Enum):
    O = 1
    Y = 2
    S = 3
    N = 4
    E = 5
    P = 6

In [ ]:
lexemas = []
for token in tokens:
    if token == 'o':
        lexemas.append({"valor" : token , "tipo": TipoToken.O })
    elif token == 'y':
        lexemas.append({"valor" : token , "tipo": TipoToken.Y})
    elif token == 'si':
        lexemas.append({"valor" : token , "tipo": TipoToken.S})
    elif token == 'entonces':
        lexemas.append({"valor" : token , "tipo": TipoToken.E})
    elif token == 'no':
        lexemas.append({"valor" : token , "tipo": TipoToken.N})
    else :
        lexemas.append({"valor" :token , "tipo": TipoToken.P })
        
lexemas

[{'valor': 'si', 'tipo': <TipoToken.S: 3>},
 {'valor': 'si', 'tipo': <TipoToken.S: 3>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'de', 'tipo': <TipoToken.P: 6>},
 {'valor': 'dia', 'tipo': <TipoToken.P: 6>},
 {'valor': 'entonces', 'tipo': <TipoToken.E: 5>},
 {'valor': 'mañana', 'tipo': <TipoToken.P: 6>},
 {'valor': 'no', 'tipo': <TipoToken.N: 4>},
 {'valor': 'es', 'tipo': <TipoToken.P: 6>},
 {'valor': 'de', 'tipo': <TipoToken.P: 6>},
 {'valor': 'noche', 'tipo': <TipoToken.P: 6>}]

## Analizador Sintactico

In [11]:
class EstadosAnalizador(Enum):
    INICIO = 1
    PRECEDENTE = 2
    ERROR1 = 3
    NEGACION = 4
    CONSECUENTE = 5
    ATOMO = 6
    DISYUNCION = 7
    CONJUNCION = 8
    ERROR2 = 9
    FIN = 10

In [ ]:
def analizar_lexemas(lexemas):
    EstadoActual = EstadosAnalizador.INICIO
    mensaje_error = ""

    for lexema in lexemas:
        if EstadoActual == EstadosAnalizador.INICIO:
            if lexema == "si":
                EstadoActual = EstadosAnalizador.PRECEDENTE
            else:
                EstadoActual = EstadosAnalizador.ERROR1
                mensaje_error = f"Error en INICIO: se esperaba 'si', se encontró '{lexema}'"
                break
        else:
            if EstadoActual == EstadosAnalizador.PRECEDENTE:
                if lexema == "no":
                    EstadoActual = EstadosAnalizador.NEGACION
                else:
                    if lexema == "es":
                        EstadoActual = EstadosAnalizador.CONSECUENTE
                    else:
                        EstadoActual = EstadosAnalizador.ERROR2
                        mensaje_error = f"Error en PRECEDENTE: se esperaba 'no' o 'es', se encontró '{lexema}'"
                        break
            else:
                if EstadoActual == EstadosAnalizador.NEGACION:
                    if lexema == "de":
                        EstadoActual = EstadosAnalizador.ATOMO
                    else:
                        EstadoActual = EstadosAnalizador.ERROR2
                        mensaje_error = f"Error en NEGACION: se esperaba 'de', se encontró '{lexema}'"
                        break
                else:
                    if EstadoActual == EstadosAnalizador.CONSECUENTE:
                        if lexema == "de":
                            EstadoActual = EstadosAnalizador.ATOMO
                        else:
                            EstadoActual = EstadosAnalizador.ERROR2
                            mensaje_error = f"Error en CONSECUENTE: se esperaba 'de', se encontró '{lexema}'"
                            break
                    else:
                        if EstadoActual == EstadosAnalizador.ATOMO:
                            if lexema == "entonces":
                                EstadoActual = EstadosAnalizador.FIN
                            else:
                                if lexema == "y":
                                    EstadoActual = EstadosAnalizador.CONJUNCION
                                else:
                                    if lexema == "o":
                                        EstadoActual = EstadosAnalizador.DISYUNCION
                                    else:
                                        EstadoActual = EstadosAnalizador.ERROR2
                                        mensaje_error = f"Error en ATOMO: se esperaba 'entonces', 'y' o 'o', se encontró '{lexema}'"
                                        break
                        else:
                            if EstadoActual == EstadosAnalizador.CONJUNCION:
                                if lexema == "de":
                                    EstadoActual = EstadosAnalizador.ATOMO
                                else:
                                    EstadoActual = EstadosAnalizador.ERROR2
                                    mensaje_error = f"Error en CONJUNCION: se esperaba 'de', se encontró '{lexema}'"
                                    break
                            else:
                                if EstadoActual == EstadosAnalizador.DISYUNCION:
                                    if lexema == "de":
                                        EstadoActual = EstadosAnalizador.ATOMO
                                    else:
                                        EstadoActual = EstadosAnalizador.ERROR2
                                        mensaje_error = f"Error en DISYUNCION: se esperaba 'de', se encontró '{lexema}'"
                                        break
                                else:
                                    if EstadoActual == EstadosAnalizador.ERROR1:
                                        mensaje_error = "Error en ERROR1: análisis detenido"
                                        break
                                    else:
                                        if EstadoActual == EstadosAnalizador.ERROR2:
                                            mensaje_error = "Error en ERROR2: análisis detenido"
                                            break
                                        else:
                                            if EstadoActual == EstadosAnalizador.FIN:
                                                break

    if EstadoActual == EstadosAnalizador.FIN:
        return True, "Análisis completado con éxito"
    else:
        return False, mensaje_error

In [13]:
lexemas = ["si", "es", "de", "entonces"]
resultado, mensaje = analizar_lexemas(lexemas)
print(resultado, mensaje)

EstadosAnalizador.ERROR1


## Arbol Binario

In [ ]:
from prettytable import PrettyTable

class Nodo: 
    def __init__(self, regla):
        self.regla = regla
        self.izquierda = None
        self.derecha = None

# Crear los nodos del árbol
nodo1 = Nodo("Si el semáforo está en rojo entonces detente.")
nodo2 = Nodo("Si el semáforo está en verde entonces avanza.")
nodo3 = Nodo("El semáforo no está en verde, debes frenar.")
nodo4 = Nodo("No hay señalización, sigue las normas generales de tránsito.")
nodo5 = Nodo("Es de noche y hay poca visibilidad por lo que debes encender las luces.")
nodo6 = Nodo("Hay un peatón y está en el paso de cebra por lo que debes detenerte.")
nodo7 = Nodo("Debes hacerte a un lado cuando hay un vehículo de emergencia o te pueden multar.")
nodo8 = Nodo("Si hay obras en la vía debes disminuir la velocidad o tendrás un accidente.")
nodo9 = Nodo("Si es una zona escolar y es horario escolar entonces disminuye la velocidad.")
nodo10 = Nodo("Si está lloviendo y la carretera está resbaladiza entonces maneja con precaución.")

# Asignar las ramas del árbol
nodo1.izquierda = nodo3
nodo1.derecha = nodo2
nodo3.izquierda = nodo4
nodo3.derecha = nodo5
nodo5.izquierda = nodo6
nodo5.derecha = nodo7
nodo7.izquierda = nodo8
nodo7.derecha = nodo9
nodo8.izquierda = nodo10

# Función para recorrer el árbol en orden
def recorrer_arbol(nodo, si_phrases, no_si_phrases):
    if nodo is not None:
        recorrer_arbol(nodo.izquierda, si_phrases, no_si_phrases)
        if nodo.regla.startswith("Si"):
            si_phrases.append(nodo.regla)
        else:
            no_si_phrases.append(nodo.regla)
        recorrer_arbol(nodo.derecha, si_phrases, no_si_phrases)

# Recorrer el árbol en orden
si_phrases = []
no_si_phrases = []
recorrer_arbol(nodo1, si_phrases, no_si_phrases)
# Imprimir las frases que empiezan con "Si"
print("Aquí están las frases que empiezan con 'Si':")
for frase in si_phrases:
    print(frase)

# Imprimir las frases que no empiezan con "Si"
print("\nAquí están las frases que no empiezan con 'Si':")
for frase in no_si_phrases:
    print(frase)

# Crear la tabla con las frases y su número de puesto
tabla = []
for i, frase in enumerate(si_phrases + no_si_phrases, start=1):
    tabla.append((frase, i))

# Usar PrettyTable para imprimir la tabla
tabla_pretty = PrettyTable()
tabla_pretty.field_names = ["Frase", "Puesto"]
for frase, puesto in tabla:
    tabla_pretty.add_row([frase, puesto])

print("\nTabla de frases:")
print(tabla_pretty)